In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import json
from pandas.io.json import json_normalize
import copy

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
feattype = 'raw'
traindf2 = pd.read_pickle('../data/processed_'+feattype+'/train_'+feattype+'_first.pickle')
test = pd.read_pickle('../data/processed_'+feattype+'/test_'+feattype+'_first.pickle')
#traindf1 = pd.read_pickle('../data/processed_'+feattype+'/train_'+feattype+'_row.pickle')
#querydf = pd.read_csv('../data/raw/data_set_phase1/train_queries.csv')
#plandf = pd.read_csv('../data/raw/data_set_phase1/train_plans.csv')


In [9]:
list(test.iloc[0])

[2848914.0,
 '2018-11-17 18:42:17',
 1.0,
 101804.0,
 116.36,
 40.07,
 116.0,
 40.35,
 0.45607017003965544,
 5.0,
 12.0,
 1.8159605922238937,
 -0.5268372454503001,
 -0.7637804039394607,
 0.04783394090855956,
 1.4397662893124945,
 0.062001101678447045,
 0.1435467993083614,
 -0.28958934312904466,
 0.054620407905024884,
 0.09180098482991404,
 -0.08728642792152969,
 -0.860515357395704,
 0.5707664971851927,
 -0.304734767498892,
 -0.09180133867828881,
 0.27965122868221276,
 -0.17597125022284496,
 0.929617369765522,
 0.007844956184059173,
 -0.47575773040327474,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 '2018-11-17 12:56:15',
 0.0,
 1.0,
 0.0,
 

In [6]:
list(traindf2.columns)

['sid',
 'click_time',
 'click_mode',
 'pid',
 'req_time',
 'o_long',
 'o_lat',
 'd_long',
 'd_lat',
 'distance_query',
 'svd_fea_0',
 'svd_fea_1',
 'svd_fea_2',
 'svd_fea_3',
 'svd_fea_4',
 'svd_fea_5',
 'svd_fea_6',
 'svd_fea_7',
 'svd_fea_8',
 'svd_fea_9',
 'svd_fea_10',
 'svd_fea_11',
 'svd_fea_12',
 'svd_fea_13',
 'svd_fea_14',
 'svd_fea_15',
 'svd_fea_16',
 'svd_fea_17',
 'svd_fea_18',
 'svd_fea_19',
 'plan_time',
 'mode_feas_0',
 'mode_feas_1',
 'mode_feas_2',
 'mode_feas_3',
 'mode_feas_4',
 'mode_feas_5',
 'mode_feas_6',
 'mode_feas_7',
 'mode_feas_8',
 'mode_feas_9',
 'mode_feas_10',
 'mode_feas_11',
 'max_dist',
 'min_dist',
 'mean_dist',
 'std_dist',
 'max_price',
 'min_price',
 'mean_price',
 'std_price',
 'max_eta',
 'min_eta',
 'mean_eta',
 'std_eta',
 'max_dist_mode',
 'min_dist_mode',
 'max_price_mode',
 'min_price_mode',
 'max_eta_mode',
 'min_eta_mode',
 'first_mode',
 'svd_mode_0',
 'svd_mode_1',
 'svd_mode_2',
 'svd_mode_3',
 'svd_mode_4',
 'svd_mode_5',
 'svd_mode

In [5]:
tr_data0 = querydf.merge(plandf, on='sid', how='left')

In [7]:
def gen_plan_feas(data, col_name='plans'):
    n = data.shape[0]
    mode_list_feas = np.zeros((n, 12))
    max_dist, min_dist, mean_dist, std_dist = np.zeros((n,)), np.zeros((n,)), np.zeros((n,)), np.zeros((n,))

    max_price, min_price, mean_price, std_price = np.zeros((n,)), np.zeros((n,)), np.zeros((n,)), np.zeros((n,))

    max_eta, min_eta, mean_eta, std_eta = np.zeros((n,)), np.zeros((n,)), np.zeros((n,)), np.zeros((n,))

    min_dist_mode, max_dist_mode, min_price_mode, max_price_mode, min_eta_mode, max_eta_mode, first_mode = np.zeros(
    (n,)), np.zeros((n,)), np.zeros((n,)), np.zeros((n,)), np.zeros((n,)), np.zeros((n,)), np.zeros((n,))
    mode_texts = []
    
    for i, plan in tqdm(enumerate(data[col_name].values)):
        try:
            cur_plan_list = json.loads(plan)
        except:
            cur_plan_list = []
        if len(cur_plan_list) == 0:
            mode_list_feas[i, 0] = 1
            first_mode[i] = 0

            max_dist[i] = -1
            min_dist[i] = -1
            mean_dist[i] = -1
            std_dist[i] = -1

            max_price[i] = -1
            min_price[i] = -1
            mean_price[i] = -1
            std_price[i] = -1

            max_eta[i] = -1
            min_eta[i] = -1
            mean_eta[i] = -1
            std_eta[i] = -1

            min_dist_mode[i] = -1
            max_dist_mode[i] = -1
            min_price_mode[i] = -1
            max_price_mode[i] = -1
            min_eta_mode[i] = -1
            max_eta_mode[i] = -1

            mode_texts.append('word_null')
        else:
            distance_list = []
            price_list = []
            eta_list = []
            mode_list = []
            for tmp_dit in cur_plan_list:
                distance_list.append(int(tmp_dit['distance']))
                if tmp_dit['price'] == '':
                    price_list.append(0)
                else:
                    price_list.append(int(tmp_dit['price']))
                eta_list.append(int(tmp_dit['eta']))
                mode_list.append(int(tmp_dit['transport_mode']))
            mode_texts.append(' '.join(['word_{}'.format(mode) for mode in mode_list]))
            distance_list = np.array(distance_list)
            price_list = np.array(price_list)
            eta_list = np.array(eta_list)
            mode_list = np.array(mode_list, dtype='int')
            mode_list_feas[i, mode_list] = 1
            distance_sort_idx = np.argsort(distance_list)
            price_sort_idx = np.argsort(price_list)
            eta_sort_idx = np.argsort(eta_list)

            max_dist[i] = distance_list[distance_sort_idx[-1]]
            min_dist[i] = distance_list[distance_sort_idx[0]]
            mean_dist[i] = np.mean(distance_list)
            std_dist[i] = np.std(distance_list)

            max_price[i] = price_list[price_sort_idx[-1]]
            min_price[i] = price_list[price_sort_idx[0]]
            mean_price[i] = np.mean(price_list)
            std_price[i] = np.std(price_list)

            max_eta[i] = eta_list[eta_sort_idx[-1]]
            min_eta[i] = eta_list[eta_sort_idx[0]]
            mean_eta[i] = np.mean(eta_list)
            std_eta[i] = np.std(eta_list)

            first_mode[i] = mode_list[0]
            max_dist_mode[i] = mode_list[distance_sort_idx[-1]]
            min_dist_mode[i] = mode_list[distance_sort_idx[0]]

            max_price_mode[i] = mode_list[price_sort_idx[-1]]
            min_price_mode[i] = mode_list[price_sort_idx[0]]

            max_eta_mode[i] = mode_list[eta_sort_idx[-1]]
            min_eta_mode[i] = mode_list[eta_sort_idx[0]]

    feature_data = pd.DataFrame(mode_list_feas)
    feature_data.columns = ['mode_feas_{}'.format(i) for i in range(12)]
    feature_data['max_dist'] = max_dist
    feature_data['min_dist'] = min_dist
    feature_data['mean_dist'] = mean_dist
    feature_data['std_dist'] = std_dist

    feature_data['max_price'] = max_price
    feature_data['min_price'] = min_price
    feature_data['mean_price'] = mean_price
    feature_data['std_price'] = std_price

    feature_data['max_eta'] = max_eta
    feature_data['min_eta'] = min_eta
    feature_data['mean_eta'] = mean_eta
    feature_data['std_eta'] = std_eta

    feature_data['max_dist_mode'] = max_dist_mode
    feature_data['min_dist_mode'] = min_dist_mode
    feature_data['max_price_mode'] = max_price_mode
    feature_data['min_price_mode'] = min_price_mode
    feature_data['max_eta_mode'] = max_eta_mode
    feature_data['min_eta_mode'] = min_eta_mode
    feature_data['first_mode'] = first_mode
    
    print('mode tfidf...')
    tfidf_enc = TfidfVectorizer(ngram_range=(1, 2))
    tfidf_vec = tfidf_enc.fit_transform(mode_texts)
    svd_enc = TruncatedSVD(n_components=10, n_iter=20, random_state=2019)
    # peta ValueError: n_components must be < n_features; got 10 >= 1
    mode_svd = svd_enc.fit_transform(tfidf_vec)
    mode_svd = pd.DataFrame(mode_svd)
    mode_svd.columns = ['svd_mode_{}'.format(i) for i in range(10)]
    data = pd.concat([data, feature_data, mode_svd], axis=1)
        
    data = data.drop([col_name], axis=1)
    return data

In [8]:
tr_data = gen_plan_feas(tr_data0)

500000it [01:27, 5737.93it/s]


mode tfidf...


In [9]:
tr_data.sample(5)

,sid,pid,req_time,o,d,plan_time,plans,mode_feas_0,mode_feas_1,mode_feas_2,...,svd_mode_0,svd_mode_1,svd_mode_2,svd_mode_3,svd_mode_4,svd_mode_5,svd_mode_6,svd_mode_7,svd_mode_8,svd_mode_9
63773,318189,135661.0,2018-10-04 09:10:26,"116.37,39.98","116.33,39.94",2018-10-04 09:10:26,"[{""distance"": 11932, ""price"": 400, ""eta"": 2544...",0.0,1.0,1.0,...,0.532261,0.023001,0.250932,-0.050634,0.230218,0.068556,0.141742,0.107279,0.254623,-0.155611
223801,404172,NaN,2018-10-05 13:16:08,"116.40,39.85","116.49,39.98",2018-10-05 13:16:08,"[{""distance"": 24777, ""price"": 600, ""eta"": 4892...",0.0,1.0,1.0,...,0.506216,-0.215757,0.177845,0.016934,-0.216213,0.490711,-0.072036,-0.113337,-0.246603,-0.071166
317071,1984421,202643.0,2018-10-14 18:17:47,"116.42,39.90","116.49,39.99",2018-10-14 18:17:47,"[{""distance"": 16132, ""price"": 700, ""eta"": 4147...",0.0,1.0,0.0,...,0.495139,-0.193172,-0.301625,0.016364,0.152216,0.066624,0.318236,0.158883,0.445396,-0.236114
488532,2241222,191435.0,2018-10-13 18:51:21,"116.41,39.92","116.66,39.88",2018-10-13 18:51:21,"[{""distance"": 24013, ""price"": 600, ""eta"": 3335...",0.0,0.0,1.0,...,0.676237,-0.257873,0.611876,-0.027701,-0.198774,-0.096992,-0.067215,0.001633,0.021575,0.001139
301756,1923700,115984.0,2018-11-13 13:57:13,"116.32,39.89","116.31,39.97",2018-11-13 13:57:13,"[{""distance"": 9410, ""price"": 400, ""eta"": 1602,...",0.0,0.0,1.0,...,0.676237,-0.257873,0.611876,-0.027701,-0.198774,-0.096992,-0.067215,0.001633,0.021575,0.001139


In [10]:
list(tr_data.columns)

['sid',
 'pid',
 'req_time',
 'o',
 'd',
 'plan_time',
 'plans',
 'mode_feas_0',
 'mode_feas_1',
 'mode_feas_2',
 'mode_feas_3',
 'mode_feas_4',
 'mode_feas_5',
 'mode_feas_6',
 'mode_feas_7',
 'mode_feas_8',
 'mode_feas_9',
 'mode_feas_10',
 'mode_feas_11',
 'max_dist',
 'min_dist',
 'mean_dist',
 'std_dist',
 'max_price',
 'min_price',
 'mean_price',
 'std_price',
 'max_eta',
 'min_eta',
 'mean_eta',
 'std_eta',
 'max_dist_mode',
 'min_dist_mode',
 'max_price_mode',
 'min_price_mode',
 'max_eta_mode',
 'min_eta_mode',
 'first_mode',
 'svd_mode_0',
 'svd_mode_1',
 'svd_mode_2',
 'svd_mode_3',
 'svd_mode_4',
 'svd_mode_5',
 'svd_mode_6',
 'svd_mode_7',
 'svd_mode_8',
 'svd_mode_9']

In [11]:
def unstack_plans(df_plans):
    df = df_plans.copy()

    df.plans = df.apply(
        lambda x: json.loads(
            '{"plans":'
            + x.plans
            + ',"sid":"'
            + str(x.sid)
            + '"'
            + ',"plan_time":'
            + '"'
            + str(x.plan_time)
            + '"}'
        ), axis=1)

    df_unstacked = json_normalize(df.plans.values, "plans", ["sid", "plan_time"])
    df_unstacked.rename({"distance": "distance_plan"}, axis=1, inplace=True)
    return df_unstacked

In [12]:
plandf = unstack_plans(plandf)

In [13]:
plandf.sid = plandf.sid.astype(int)

In [14]:
traindf = pd.merge(plandf, tr_data, on="sid")

In [15]:
traindf.sample(5)

,distance_plan,eta,price,transport_mode,sid,plan_time_x,pid,req_time,o,d,...,svd_mode_0,svd_mode_1,svd_mode_2,svd_mode_3,svd_mode_4,svd_mode_5,svd_mode_6,svd_mode_7,svd_mode_8,svd_mode_9
1618434,32771,4458,600,2,2867657,2018-11-17 14:11:18,112198.0,2018-11-17 14:11:18,"116.34,40.03","116.41,39.84",...,0.292249,-0.134339,0.416909,0.020276,-0.084650,-0.039577,-0.006471,0.032220,-0.008067,0.043942
1257951,43535,4065,10000,4,2921342,2018-10-06 14:38:42,197112.0,2018-10-06 14:38:42,"116.84,40.38","116.65,40.13",...,0.546549,-0.289093,-0.478986,0.110851,-0.208016,-0.052177,-0.144700,0.101206,0.380259,0.178321
1757124,22535,3744,1800,10,1924323,2018-11-13 09:12:42,NaN,2018-11-13 09:12:42,"116.32,40.04","116.37,39.93",...,0.201117,-0.093572,-0.236014,0.033828,-0.085705,0.110230,-0.058525,0.008447,0.032158,0.007354
853837,244,56,,3,2248021,2018-10-13 14:07:42,164979.0,2018-10-13 14:07:42,"116.37,39.91","116.37,39.91",...,0.221682,0.444884,0.019677,0.242715,0.073740,0.085751,-0.197311,0.649472,-0.149908,-0.032120
388955,13126,3167,400,2,150243,2018-10-20 11:40:44,NaN,2018-10-20 11:40:44,"116.43,39.92","116.40,40.01",...,0.380470,-0.271813,-0.041088,0.173216,0.176911,0.311776,0.132809,-0.049984,-0.213885,0.230918


In [17]:
traindf.shape

(2276822, 53)

In [16]:
list(traindf.columns)

['distance_plan',
 'eta',
 'price',
 'transport_mode',
 'sid',
 'plan_time_x',
 'pid',
 'req_time',
 'o',
 'd',
 'plan_time_y',
 'plans',
 'mode_feas_0',
 'mode_feas_1',
 'mode_feas_2',
 'mode_feas_3',
 'mode_feas_4',
 'mode_feas_5',
 'mode_feas_6',
 'mode_feas_7',
 'mode_feas_8',
 'mode_feas_9',
 'mode_feas_10',
 'mode_feas_11',
 'max_dist',
 'min_dist',
 'mean_dist',
 'std_dist',
 'max_price',
 'min_price',
 'mean_price',
 'std_price',
 'max_eta',
 'min_eta',
 'mean_eta',
 'std_eta',
 'max_dist_mode',
 'min_dist_mode',
 'max_price_mode',
 'min_price_mode',
 'max_eta_mode',
 'min_eta_mode',
 'first_mode',
 'svd_mode_0',
 'svd_mode_1',
 'svd_mode_2',
 'svd_mode_3',
 'svd_mode_4',
 'svd_mode_5',
 'svd_mode_6',
 'svd_mode_7',
 'svd_mode_8',
 'svd_mode_9']

In [28]:
def create_new_plans(r):
    newjson = '{"plans":'
    newlist = json.loads(r.plans)
    tmp = copy.deepcopy(newlist)
    for plan_info in newlist:
        plan_info['mode_plan'] = tmp
    newjson += json.dumps(newlist)
    newjson += ',"sid":"' + str(r.sid) + '","plan_time":"' + str(r.plan_time) + '"}'
    return json.loads(newjson)

In [ ]:
df = plandf.copy()
df.plans = df.apply(create_new_plans, axis=1)

In [7]:
df.plans.iloc[0]

{'plans': [{'distance': 32303,
   'price': 600,
   'eta': 5087,
   'transport_mode': 9,
   'mode_plan': {'distance': 32303,
    'price': 600,
    'eta': 5087,
    'transport_mode': 9}},
  {'distance': 33678,
   'price': '',
   'eta': 3149,
   'transport_mode': 3,
   'mode_plan': {'distance': 33678,
    'price': '',
    'eta': 3149,
    'transport_mode': 3}},
  {'distance': 33678,
   'price': 10000,
   'eta': 3269,
   'transport_mode': 4,
   'mode_plan': {'distance': 33678,
    'price': 10000,
    'eta': 3269,
    'transport_mode': 4}},
  {'distance': 32099,
   'price': 600,
   'eta': 5996,
   'transport_mode': 2,
   'mode_plan': {'distance': 32099,
    'price': 600,
    'eta': 5996,
    'transport_mode': 2}},
  {'distance': 30446,
   'price': 900,
   'eta': 6494,
   'transport_mode': 1,
   'mode_plan': {'distance': 30446,
    'price': 900,
    'eta': 6494,
    'transport_mode': 1}}],
 'sid': '1709112',
 'plan_time': '2018-11-04 11:45:04'}

In [8]:
df_unstacked = json_normalize(df.plans.values, "plans", ["sid", "plan_time"])

In [9]:
df_unstacked.iloc[0]

distance                                                      32303
eta                                                            5087
mode_plan         {'distance': 32303, 'price': 600, 'eta': 5087,...
price                                                           600
transport_mode                                                    9
sid                                                         1709112
plan_time                                       2018-11-04 11:45:04
Name: 0, dtype: object

In [10]:
df_unstacked.sid = df_unstacked.sid.astype(int)

In [11]:
df_unstacked.shape

(2276822, 7)

In [12]:
df_unstacked.rename({"distance": "distance_plan"}, axis=1, inplace=True)

In [13]:
df_unstacked.sample(5)

,distance_plan,eta,mode_plan,price,transport_mode,sid,plan_time
67991,943,282,"{'distance': 943, 'price': '', 'eta': 282, 'tr...",,6,1187613,2018-10-15 15:04:41
2091597,15589,1474,"{'distance': 15589, 'price': '', 'eta': 1474, ...",,3,131823,2018-10-08 02:11:08
2174960,44509,7765,"{'distance': 44509, 'price': 900, 'eta': 7765,...",900,7,1845931,2018-11-03 21:46:37
2044962,34390,6488,"{'distance': 34390, 'price': 900, 'eta': 6488,...",900,1,1053599,2018-11-27 19:19:17
410056,1551,371,"{'distance': 1551, 'price': '', 'eta': 371, 't...",,3,2539360,2018-11-19 21:33:14


In [14]:
tr_data1 =  pd.merge(df_unstacked, querydf, on="sid")

In [15]:
tr_data1.iloc[0]

distance_plan                                                 32303
eta                                                            5087
mode_plan         {'distance': 32303, 'price': 600, 'eta': 5087,...
price                                                           600
transport_mode                                                    9
sid                                                         1709112
plan_time                                       2018-11-04 11:45:04
pid                                                             NaN
req_time                                        2018-11-04 11:45:04
o                                                      116.30,39.84
d                                                      116.47,39.98
Name: 0, dtype: object

In [19]:
tr_data2 = gen_plan_feas(tr_data1, col_name='mode_plan')

2276822it [00:07, 325182.92it/s]


In [20]:
tr_data2.sample(5)

,distance_plan,eta,price,transport_mode,sid,plan_time,pid,req_time,o,d,...,min_eta,mean_eta,std_eta,max_dist_mode,min_dist_mode,max_price_mode,min_price_mode,max_eta_mode,min_eta_mode,first_mode
1497953,4619,1519,1700,4,2981201,2018-11-02 19:27:35,188009.0,2018-11-02 19:27:35,"116.39,39.98","116.39,39.99",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
34568,846,253,,6,1881671,2018-11-03 12:44:27,207313.0,2018-11-03 12:44:27,"116.47,39.99","116.48,39.99",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
720731,21338,4258,700,7,96089,2018-10-31 16:17:20,161605.0,2018-10-31 16:17:20,"116.36,40.10","116.39,39.95",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
1697615,17296,4247,500,2,1612293,2018-10-03 08:05:41,192172.0,2018-10-03 08:05:41,"116.41,39.97","116.36,39.87",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
525732,17856,2988,500,2,1943609,2018-10-11 20:23:55,169530.0,2018-10-11 20:23:55,"116.39,39.90","116.21,39.90",...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
